In [92]:
# Write a CSV of GTFS stops served by specified routes
# Requires a config.yaml with 
#   feed: (GTFS feed), 
#   route_ids: (string or integer series), 
#   agency + route_short_names (if route_ids are inconvenient to use)

import partridge as pa
import pandas as pd
import yaml

# Interactive input
folder = input('Enter config folder within data directory: ')
dir = 'data/' + folder + '/'

# Load config file
config = yaml.load(open(dir + 'config.yaml'))

#Load feed and tables
feed = pa.load_raw_feed(dir + config.get('feed'))
r = feed.routes
t = feed.trips
st = feed.stop_times
s = feed.stops

route_ids = pd.DataFrame({'route_id' : list(map(str, config.get('route_ids')))})

agency_id = config.get('agency_id')
if (agency_id):
    agency_id = str(agency_id)

route_short_names = config.get('route_short_names')
if (route_short_names): 
    route_short_names = list(map(str, route_short_names))
    
# Combine route lists
if (isinstance(route_short_names, list) & isinstance(agency_id, str)):
    route_ids = pd.DataFrame(
        r[((r['route_short_name'].isin(route_short_names)) & (r['agency_id'] == agency_id))] \
        ['route_id']) \
        .append(route_ids)

# Look up relevant trips and stops
trip_ids = t[t['route_id'].isin(route_ids.iloc[:,0] )]['trip_id']
stop_ids = st[st['trip_id'].isin(trip_ids)]['stop_id']
stops = s[s['stop_id'].isin(stop_ids)].loc[:,['stop_id','stop_lat','stop_lon']]

# Write stops
stops.to_csv('data/stops_' + folder + '.csv', index=False)
print("Done writing.")

Enter config folder within data directory: calgary-2019
Done writing.
